2016-10-20일 뉴스에서 하나의 뉴스와 비슷한 뉴스를 찾아보겠습니다. 이를 위하여 Day5_3에서 만들어둔, 당일의 명사로 이뤄진 term frequency matrix를 이용합니다. 

In [3]:
mm_fname = '../../../data/corpus_10days/models/2016-10-20_noun_x.mm'
corpus_fname = '../../../data/corpus_10days/news/2016-10-20_article_all_normed.txt'

mmread(fname)의 return type은 coo_matrix이며, 이는 slice가 안됩니다. 그렇기 때문에 csr_matrix로 바꿔줘야 합니다. 

x.shape과 len(docs)의 길이가 맞는지도 확인합니다. 

In [4]:
from scipy.io import mmread
x = mmread(mm_fname).tocsr()

with open(corpus_fname, encoding='utf-8') as f:
    docs = [doc.strip() for doc in f]

print(x.shape, len(docs))

(30091, 2631) 30091


임의의 문서 하나를 선택하였습니다. 미르재단과 최순실과의 관계에 대한 뉴스입니다.

이 뉴스의 term frequency vector는 x[2317,:]으로 선택할 수 있습니다. 

In [5]:
docs[2317]

'서울 연합뉴스 비선 실세 논란의 중심에 선 최순실씨를 둘러싼 각종 의혹이 난무하고 있다 전경련이 800억 원 가까운 기금을 내 만든 문화 스포츠 재단인 미르와 스포츠 설립 및 인사 관련 논란으로 출발한 최순실씨의 재단 사유화 의혹은 정경유착 논쟁으로 번지면서 최대 재계 단체인 전국경제인연합회의 해체 압력으로 이어졌고 최씨 딸의 이화여대 체육특기자 입학과 성적 등 학사관리를 둘러싼 특혜 의혹은 최경희 총장의 사퇴를 부르는 등 파문이 확산되고 있다 최씨가 사실상의 소유주라는 의심을 받는 더블루케이와 최씨 모녀의 개인회사인 비텍스포츠는 스포츠재단을 통해 들어온 돈을 사적으로 쓰려 했다는 보도까지 나왔다 여기에 최씨 모녀의 부적절한 언동이 를 타고 퍼지면서 여론이 악화하고 있다  청와대는 미르 스포츠재단 기금 모금과 관련한 청와대 연루설에 대해서는 이미 사실무근 이라고 부인하면서 야권의 부당한 정치 공세라고 반박한 바 있다 전경련도 미르와 스포츠는 기업들이 자발적으로 설립한 재단으로 권력의 개입은 없었다고 밝혔다 하지만 의혹이 수그러들기는커녕 갈수록 가지를 뻗고 있어 이 문제를 그냥 넘길 수 없는 상황이 됐다 여권 내부에서조차 비주류를 중심으로 권력형 비리 파문으로 확산하는 것을 차단하기 위해서라도 검찰 수사 등을 통해 신속히 진상을 규명해야 한다는 목소리가 커지고 있다 일각에서는 미르 스포츠재단 문제가 블랙홀처럼 모든 현안을 빨아들이는 판에 집권 여당이 진상 규명을 막는 듯한 모습을 보이는 것은 향후 국정 운영에 부담이 될 것이라고 우려한다 야권에서는 지금까지 불거진 최씨와 관련한 각종 의혹을 최순실 게이트 로 규정하고 국정조사와 특검까지 거론하고 있다  박근혜 대통령은 이와 관련 20일 수석비서관회의에서 어느 누구라도 재단과 관련해 자금 유용 등 불법 행위를 저질렀다면 엄정하게 처벌받을 것 이라고 밝혔다 한 달 전 미르 스포츠재단 의혹이 불거진 이후 박 대통령이 이처럼 구체적으로 언급한 건 처음이다 박 대통령은 문화 체육 분야를 집중 지원하고 우리 문화를 알리며

앞서 연습하였던 LSHForest를 이용하여 Cosine distance 기준으로 유사한 문서를 찾겠습니다. training은 lsh.fit(x)로 끝납니다. 패러메터는 기본값을 이용하였습니다. 

In [6]:
%%time

from sklearn.neighbors import LSHForest

lsh = LSHForest()
lsh.fit(x)

CPU times: user 608 ms, sys: 36 ms, total: 644 ms
Wall time: 644 ms


2317번 문서와 유사한 10개의 문서를 선택하였습니다. 가장 비슷한 문서는 2317번이어야 합니다. 

In [15]:
dist, idxs = lsh.kneighbors(x[2317:2319,:], n_neighbors=10)

In [16]:
idxs.tolist()

[[2317, 10772, 28771, 12637, 9091, 11762, 12457, 24611, 2203, 1320],
 [2318, 29990, 1471, 1472, 2660, 231, 1451, 175, 1009, 1012]]

In [18]:
idxs.tolist()[0]

[2317, 10772, 28771, 12637, 9091, 11762, 12457, 24611, 2203, 1320]

In [9]:
dist[0]

array([  1.33226763e-15,   2.74315827e-01,   2.76449185e-01,
         2.91295308e-01,   3.00332700e-01,   3.03354187e-01,
         3.29854402e-01,   3.39813581e-01,   3.42880902e-01,
         3.56541417e-01])

2317번을 포함한 10개의 문서에 대하여 각 문서 별 200글자까지만 snippest를 출력하여 그 내용을 확인하겠습니다. 이와 같이 비슷한 문서를 찾는 문제에서는 LSHForest를 쓰면 편리합니다. 

또한, 문서는 어느 정도 길이가 긴 문서인 것이 좋습니다. 짧은 문서는 too much sparse하여 인덱싱이 잘 되지 않을 수 있습니다. 

In [20]:
for similar_doc in idxs[0]:
    print('%s ...\n' % docs[similar_doc][:100])

서울 연합뉴스 비선 실세 논란의 중심에 선 최순실씨를 둘러싼 각종 의혹이 난무하고 있다 전경련이 800억 원 가까운 기금을 내 만든 문화 스포츠 재단인 미르와 스포츠 설립 및 인사 ...

검찰에 엄정 수사 촉구 미르 스포츠재단 감독강화 지시  모두발언 17분중 9분 할애 안타깝다 사심없다 심경 토로  이데일리 이준기 기자 미르 케이 스포츠 의혹의 핵심당사자이자 현  ...

청와대 1  미르 첫 보도 3개월 첫 무대응 반응 한달 만  정치권분열 의혹확산 국정신뢰 국감 하루 앞  서울 뉴스1 윤태형 기자 미르 케이 스포츠 재단 의혹이 이른 바 비선실세  ...

한겨레 검찰 수사로 정권 불똥 막기  의미있는 사업에 인신공격  강제모금 초고속 설립 모르쇠  퇴임 대비한 재단 의혹 부인  박근혜 대통령이 20일 오후 청와대에서 열린 수석비서관 ...

불법 있으면 엄중 처벌  박 대통령 엄정 수사 메시지 검찰 최순실 통화내역 조회  국정 운영 발목 잡힐라 선회 박 대통령 퇴임 후 대비 아니다  미르 재단 문화융성 위해 기업들이  ...

수석회의 모두 발언 절반 할애 기업들 자발적으로 자금 모금 검찰에 수사 가이드라인 제시 최씨 재단설립 개입 여부 언급 안해 야 공세에 인신공격성 논란 규정 사실상 최순실 감싸기 지 ...

박근혜 대통령이 어제 미르 스포츠재단 의혹과 관련해 저의 퇴임 후를 대비해 만들어졌다는데 그럴 이유도 없고 사실도 아니다 고 부인했다 이어 만약 어느 누구라도 재단과 관련해 자금  ...

모두 발언 18분중 절반 이상을 재단과 비선실세 의혹 해명  역대 최저지지율에 국정 부담  권력형 아닌 개인비리 해석  박근혜 대통령이 20일 청와대에서 수석비서관회의를 주재한 자 ...

박 대통령 수석비서관회의 발언 서울 연합뉴스 백승렬 기자 박근혜 대통령이 20일 오후 청와대에서 열린 수석비서관회의에서 발언하고 있다  수석회의서 미르 스포츠 재단 설명하며 철저수 ...

미르 스포츠재단 사유화 시도 의혹 본격수사 관심  서울 연합뉴스 차대운 박경준 기자